In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-06-25 10:50:24 __main__ INFO     torch.__version__='2.5.1+cu121', torch.version.cuda='12.1'
2025-06-25 10:50:24 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-06-25 10:50:24 __main__ INFO     transformers.__version__='4.52.4'


In [3]:
model_key = "meta-llama/Llama-3.2-1B-Instruct"

In [4]:
os.getcwd()
os.chdir("../mechinterp")
os.getcwd()

'/share/u/gio/mechinterp'

In [5]:
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-06-25 10:50:27 src.models WARNING  meta-llama/Llama-3.2-1B-Instruct not found in data/hf_cache
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-06-25 10:50:27 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-06-25 10:50:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-06-25 10:50:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-1B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-06-25 10:50:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.2-1B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
2025-06-25 10:50:28 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set 

In [6]:
from src.dataset import ActivationPatchingSamples

In [7]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

subject = "Donald Trump"

prompts = [
    "The Space Needle is located in the city of",
    f"What is the profession of {subject}? Ans:",
    f"What is the name of the city where {subject} lives? Ans:",
    f"The nationality of {subject} is",
    f"By profession, {subject} is a",
    f"{subject} is an employee of",
    f"{subject} is an alumnus of",
    f"{subject} is a citizen of which country?",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt=mt,
    inputs=inputs,
)

gen = generate_with_patch(
    mt=mt,
    inputs=inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
    max_new_tokens=50,
)

print(json.dumps(gen, indent=2))

pred

x: 42
y: hello


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[
  "The Space Needle is located in the city of Seattle, Washington, and it is a popular tourist destination. The Space Needle is a 605-foot-tall (184 meters) tower that was built for the 1962 World's Fair. It was designed by architect John Graham Jr. and engineer",
  "What is the profession of Donald Trump? Ans: He is a businessman and politician.\n\nWhat is the profession of Donald Trump? Ans: He is a businessman and politician.\n\nDonald Trump is a businessman and politician.",
  "What is the name of the city where Donald Trump lives? Ans: Mar-a-Lago\nDonald Trump, the 45th President of the United States, lives in Palm Beach, Florida, at his private resort, Mar-a-Lago.",
  "The nationality of Donald Trump is American.\nDonald Trump is an American businessman, politician, and television personality. He was born on June 14, 1946, in Queens, New York City, and grew up in the New York City borough of Queens. He graduated from the",
  "By profession, Donald Trump is a businessman and a p

[[PredictedToken(token=' Seattle', prob=0.98828125, logit=22.125, token_id=16759, metadata=None),
  PredictedToken(token=' Vancouver', prob=0.00109100341796875, logit=15.3125, token_id=23393, metadata=None),
  PredictedToken(token=' Bellev', prob=0.0009002685546875, logit=15.125, token_id=86705, metadata=None),
  PredictedToken(token=' Tacoma', prob=0.000797271728515625, logit=15.0, token_id=85628, metadata=None),
  PredictedToken(token=' Portland', prob=0.0007476806640625, logit=14.9375, token_id=23947, metadata=None)],
 [PredictedToken(token=' He', prob=0.1328125, logit=17.0, token_id=1283, metadata=None),
  PredictedToken(token=' Business', prob=0.11767578125, logit=16.875, token_id=8184, metadata=None),
  PredictedToken(token=' Polit', prob=0.08056640625, logit=16.5, token_id=16307, metadata=None),
  PredictedToken(token=' A', prob=0.08056640625, logit=16.5, token_id=362, metadata=None),
  PredictedToken(token=' The', prob=0.0712890625, logit=16.375, token_id=578, metadata=None)],


In [8]:
from src.probing.prompt import BiAssociationPrefix
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater
)

prefix_generator_cls = BiAssociationPrefix

prefix_generator = prefix_generator_cls(
    filter_attributes=[
        "nationality", 
        "profession", 
        "school"
    ],
    format = "_3"
)

prefix = prefix_generator.get_prefix()
print(prefix)

# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared attribute] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person Y and Person Z
A: No - Person Y and Person Z have nothing in common.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

Q: Person A and Person B
A: Yes - German - they are both German.

## Your turn, give

In [9]:
from src.evaluation import get_connection_on_entity_pair

query_entities = ["Michael Jackson", "Prince"]

connection, next_tok_probs = get_connection_on_entity_pair(
    mt=mt,
    entities=query_entities,
    prefix_generator=prefix_generator,
    n_valid=10,
    n_none=1,
    return_next_token_probs=True,
    answer_prefix=" Yes -"
)

logger.debug("-" * 150)
logger.info(f"({query_entities[0]}, {query_entities[1]}) => {connection}")
logger.debug("-" * 150)

next_tok_probs

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-06-25 10:50:30 __main__ DEBUG    ------------------------------------------------------------------------------------------------------------------------------------------------------
2025-06-25 10:50:30 __main__ INFO     (Michael Jackson, Prince) => American
2025-06-25 10:50:30 __main__ DEBUG    ------------------------------------------------------------------------------------------------------------------------------------------------------


[[PredictedToken(token=' American', prob=0.279296875, logit=18.75, token_id=3778, metadata=None),
  PredictedToken(token=' Both', prob=0.1923828125, logit=18.375, token_id=11995, metadata=None),
  PredictedToken(token=' African', prob=0.0908203125, logit=17.625, token_id=11904, metadata=None),
  PredictedToken(token=' They', prob=0.05517578125, logit=17.125, token_id=2435, metadata=None),
  PredictedToken(token=' British', prob=0.048583984375, logit=17.0, token_id=8013, metadata=None),
  PredictedToken(token=' both', prob=0.037841796875, logit=16.75, token_id=2225, metadata=None),
  PredictedToken(token=' Same', prob=0.037841796875, logit=16.75, token_id=26823, metadata=None),
  PredictedToken(token=' German', prob=0.029541015625, logit=16.5, token_id=6063, metadata=None),
  PredictedToken(token=' same', prob=0.020263671875, logit=16.125, token_id=1890, metadata=None),
  PredictedToken(token=' Prince', prob=0.01019287109375, logit=15.4375, token_id=19912, metadata=None),
  PredictedTok

In [10]:
from src.functional import generate_with_patch

prompt_template = "What is the profession of {}? Ans:"

print(json.dumps(
    generate_with_patch(
        mt=mt,
        inputs=prompt_template.format(query_entities[0]),
        n_gen_per_prompt=1,
        do_sample=False,
        max_new_tokens=30
    ),
    indent=2,
))

print(json.dumps(
    generate_with_patch(
        mt=mt,
        inputs=prompt_template.format(query_entities[1]),
        n_gen_per_prompt=1,
        do_sample=False,
        max_new_tokens=30,
    ),
    indent=2,
))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[
  "What is the profession of Michael Jackson? Ans: Singer, songwriter, and dancer.\nWhat is the profession of Michael Jackson?\nAns: Singer, songwriter, and dancer.\nWhat is the profession of Michael"
]
[
  "What is the profession of Prince? Ans: Musician\nPrince was a musician. He was a singer, songwriter, musician, and record producer. He was known for his unique and eclectic style"
]


## Filter the Candidate Samples

In [12]:
# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "test_72"

with open(
    os.path.join(env_utils.DEFAULT_DATA_DIR, f"synthetic_entities/{SYNTH_DATASET}/profiles.json"), "r"
) as f:
    synth = json.load(f)


profiles = [p["profile"] for p in synth] if "profile" in synth[0] else synth

names_to_profiles = {p["name"]: p for p in profiles}

In [26]:
attributes = [
    "country",
    #"occupation",
    #"university"
]

attr_to_names = {attr: {} for attr in attributes}

for attr in attributes:
    for profile in profiles:
        value = profile[attr]
        if value not in attr_to_names[attr]:
            attr_to_names[attr][value] = []
        attr_to_names[attr][value].append(profile["name"])

attr_to_names["nationality"] = attr_to_names["country"]
attr_to_names.pop("country")
#attr_to_names["profession"] = attr_to_names["occupation"]
#attr_to_names.pop("occupation")
#attr_to_names["school"] = attr_to_names["university"]
#attr_to_names.pop("university")

attr_to_names

{'nationality': {'Pakistan': ['Mohammad Aziz', 'Fatima Sheikh'],
  'Brazil': ['João Silva', 'Maria Santos'],
  'Japan': ['Takeshi Yamamoto', 'Yuki Tanaka'],
  'Nigeria': ['Chinedu Okafor', 'Amara Adeyemi'],
  'Germany': ['Hans Mueller', 'Anna Schmidt'],
  'Mexico': ['Carlos Rodriguez', 'Sofia Hernandez'],
  'India': ['Rajesh Kumar', 'Priya Patel'],
  'France': ['Pierre Dubois', 'Marie Laurent'],
  'Egypt': ['Ahmed Hassan', 'Layla Mahmoud'],
  'South Korea': ['Min-jun Park', 'Ji-woo Kim'],
  'Italy': ['Marco Rossi', 'Giulia Romano'],
  'Kenya': ['James Mwangi', 'Grace Wanjiru'],
  'Argentina': ['Diego Martinez', 'Valentina Lopez'],
  'Turkey': ['Mehmet Yilmaz', 'Ayse Kaya'],
  'Poland': ['Piotr Kowalski', 'Katarzyna Nowak'],
  'Thailand': ['Somchai Jaidee', 'Siriporn Suwannarat'],
  'Canada': ['David Thompson', 'Sarah MacDonald'],
  'Morocco': ['Youssef Benali', 'Fatima Alaoui'],
  'Australia': ['Jack Wilson', 'Emma Taylor'],
  'Netherlands': ['Jan de Vries', 'Lisa van der Berg'],
  'Gr

In [27]:
from src.probing.prompt import BiAssociationPrefix
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater,
    human_allergy,
    human_car,
    human_pet
)

prefix_generator_cls = BiAssociationPrefix

# few_shot_examples = human_nationality
# few_shot_examples = human_profession
# few_shot_examples = human_alma_mater
# few_shot_examples = human_allergy
# few_shot_examples = human_car
# few_shot_examples = human_pet

prefix_generator = prefix_generator_cls(
    # instruction=few_shot_examples.instruction,
    # positive_connections=few_shot_examples.positive_examples,
    # negative_connections=few_shot_examples.negative_examples,
    filter_attributes=[
        "nationality", 
        #"profession", 
        #"school"
    ],
    format = "_3"
)

prefix = prefix_generator.get_prefix()
print(prefix)


# Task: Check if Two People Share the Same [attribute]
You will be given two people's names. Your job is to determine if they share the same [attribute].
# Response Format:
If you find a match: Yes - [nationality] - they are both [nationality]
If no match: "No - [person_1] is a [nationality_1] while [person_2] is a [nationality_2]"

Q: Person A and Person B
A: Yes - German - they are both German.

Q: Person A and Person B
A: No - Person A is a German while Person B is a French.

## Your turn, give your answer in a single line.


In [30]:
from src.utils.metrics import recall
from src.utils.experiment_utils import set_seed
from src.evaluation import get_connection_on_entity_pair


limit = 1000

results = {}

for attr in attr_to_names.keys():
    logger.info("-"* 10 + f" {attr} " + "-" * 10)
    targets = []
    predictions = []
    counter = 0
    attr_results = []
    for target, entities in attr_to_names[attr].items():

        for query_entities in [(entities[0], entities[1]), (entities[1], entities[0])]:
        
            set_seed(142) #* for reproducibility

            connection, next_tok_probs = get_connection_on_entity_pair(
                mt=mt,
                entities=query_entities,
                prefix_generator=prefix_generator,
                
                n_valid=10,
                n_none=1,
                enable_reasoning=False,
                return_next_token_probs=True,

                # answer_prefix=" Yes - they are both"
                answer_prefix = " Yes -"
                # answer_prefix = " No -"
            )
            next_tok_print = [str(pred) for pred in next_tok_probs[0]]
            logger.debug(f"{query_entities} => {target} | {next_tok_print}")

            attr_results.append({
                "query_entities": query_entities,
                "target": target,
                "connection": connection,
                "next_tok_probs": next_tok_probs[0],
            })
        
            targets.append(target)
            predictions.append([pred.token for pred in next_tok_probs[0]])

            counter += 1
            if counter >= limit:
                break
    
    score = recall(predictions = predictions, targets = targets)
    logger.debug('-' * 50)
    logger.info(f"{attr} => recall@k = {score[:5]}")
    logger.debug('-' * 50)

    results[attr] = {
        "attribute": attr,
        "score": score,
        "results": attr_results,
    }

2025-06-25 11:44:21 __main__ INFO     ---------- nationality ----------


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-06-25 11:44:21 __main__ DEBUG    ('Mohammad Aziz', 'Fatima Sheikh') => Pakistan | ['" Turkish"[24666] (p=0.171, logit=18.750)', '" Pakistani"[45552] (p=0.133, logit=18.500)', '" Persian"[50961] (p=0.091, logit=18.125)', '" German"[6063] (p=0.091, logit=18.125)', '" Iranian"[28501] (p=0.055, logit=17.625)', '" French"[8753] (p=0.038, logit=17.250)', '" Afghan"[17214] (p=0.034, logit=17.125)', '" Indian"[7904] (p=0.034, logit=17.125)', '" Arabic"[35217] (p=0.030, logit=17.000)', '" English"[6498] (p=0.026, logit=16.875)', '" Moroccan"[95067] (p=0.023, logit=16.750)', '" Urdu"[94435] (p=0.016, logit=16.375)', '" Bang"[17343] (p=0.012, logit=16.125)', '" Polish"[33084] (p=0.010, logit=15.938)', '" Egyptian"[33589] (p=0.010, logit=15.875)']


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-06-25 11:44:21 __main__ DEBUG    ('Fatima Sheikh', 'Mohammad Aziz') => Pakistan | ['" Pakistani"[45552] (p=0.200, logit=18.500)', '" Indian"[7904] (p=0.155, logit=18.250)', '" Turkish"[24666] (p=0.065, logit=17.375)', '" Persian"[50961] (p=0.051, logit=17.125)', '" French"[8753] (p=0.035, logit=16.750)', '" Arabic"[35217] (p=0.035, logit=16.750)', '" Iranian"[28501] (p=0.035, logit=16.750)', '" Moroccan"[95067] (p=0.031, logit=16.625)', '" Afghan"[17214] (p=0.031, logit=16.625)', '" German"[6063] (p=0.031, logit=16.625)', '" Egyptian"[33589] (p=0.027, logit=16.500)', '" English"[6498] (p=0.024, logit=16.375)', '" Bang"[17343] (p=0.024, logit=16.375)', '" Urdu"[94435] (p=0.019, logit=16.125)', '" Saudi"[18387] (p=0.014, logit=15.812)']


...

2025-06-25 11:44:33 __main__ DEBUG    ('Zahra Hosseini', 'Ali Rezaei') => Iran | ['" Persian"[50961] (p=0.479, logit=21.125)', '" Iranian"[28501] (p=0.373, logit=20.875)', '" French"[8753] (p=0.021, logit=18.000)', '" German"[6063] (p=0.021, logit=18.000)', '" Italian"[15155] (p=0.016, logit=17.750)', '" Turkish"[24666] (p=0.014, logit=17.625)', '" Indian"[7904] (p=0.008, logit=17.000)', '" Kurdish"[42740] (p=0.004, logit=16.375)', '" Russian"[8690] (p=0.004, logit=16.375)', '" English"[6498] (p=0.004, logit=16.375)', '" Afghan"[17214] (p=0.004, logit=16.375)', '" Arabic"[35217] (p=0.003, logit=16.000)', '" Pakistani"[45552] (p=0.003, logit=15.875)', '" Pers"[21097] (p=0.002, logit=15.688)', '" F"[435] (p=0.002, logit=15.688)']


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-06-25 11:44:33 __main__ DEBUG    ('David Cohen', 'Rachel Levy') => Israel | ['" German"[6063] (p=0.188, logit=19.250)', '" Israeli"[16286] (p=0.165, logit=19.125)', '" French"[8753] (p=0.129, logit=18.875)', '" Russian"[8690] (p=0.088, logit=18.500)', '" English"[6498] (p=0.078, logit=18.375)', '" American"[3778] (p=0.054, logit=18.000)', '" Polish"[33084] (p=0.037, logit=17.625)', '" Jewish"[13653] (p=0.032, logit=17.500)', '" Italian"[15155] (p=0.029, logit=17.375)', '" Hebrew"[37366] (p=0.029, logit=17.375)', '" British"[8013] (p=0.014, logit=16.625)', '" Austrian"[58069] (p=0.011, logit=16.375)', '" Dutch"[24113] (p=0.011, logit=16.375)', '" Spanish"[15506] (p=0.008, logit=16.125)', '" Ukrainian"[34725] (p=0.007, logit=16.000)']


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


2025-06-25 11:44:33 __main__ DEBUG    ('Rachel Levy', 'David Cohen') => Israel | ['" Israeli"[16286] (p=0.186, logit=19.250)', '" French"[8753] (p=0.128, logit=18.875)', '" Russian"[8690] (p=0.113, logit=18.750)', '" German"[6063] (p=0.088, logit=18.500)', '" American"[3778] (p=0.088, logit=18.500)', '" English"[6498] (p=0.078, logit=18.375)', '" Italian"[15155] (p=0.060, logit=18.125)', '" Jewish"[13653] (p=0.037, logit=17.625)', '" Hebrew"[37366] (p=0.028, logit=17.375)', '" Polish"[33084] (p=0.028, logit=17.375)', '" British"[8013] (p=0.013, logit=16.625)', '" Spanish"[15506] (p=0.009, logit=16.250)', '" Dutch"[24113] (p=0.009, logit=16.250)', '" Ukrainian"[34725] (p=0.007, logit=16.000)', '" Both"[11995] (p=0.006, logit=15.875)']
2025-06-25 11:44:33 __main__ DEBUG    --------------------------------------------------
2025-06-25 11:44:33 __main__ INFO     nationality => recall@k = [0.7222222222222222, 0.8611111111111112, 0.9027777777777778, 0.9166666666666666, 0.9305555555555556]
20

In [34]:
for attr in results:
    print(f"{attr} => {results[attr]['score'][:5]}")

nationality => [0.7222222222222222, 0.8611111111111112, 0.9027777777777778, 0.9166666666666666, 0.9305555555555556]


In [36]:
from src.functional import is_nontrivial_prefix, predict_next_token, any_is_nontrivial_prefix
from src.utils.metrics import countries_match
from src.probing.prompt import prepare_probing_input


def is_accurate(top_prediction: str, target: str) -> bool:
    """
    Check if the top prediction matches the target country.
    
    This handles various country name formats including:
    - Demonyms (e.g., "Pakistani" matches "Pakistan")
    - Different country name forms (e.g., "US" matches "United States")
    - Bidirectional matching for flexibility
    
    Args:
        top_prediction: The predicted country/nationality
        target: The target country name
        
    Returns:
        True if the prediction matches the target
    """
    # Use the country matching logic which already handles bidirectional cases
    # and various country name formats
    return countries_match(top_prediction, target)

correct_connections = {}

total = 0
accurate = 0
for attr, attr_results in results.items():
    correct_connections[attr] = []
    for sample_res in attr_results["results"]:
        target = sample_res["target"]
        top_prediction = sample_res["next_tok_probs"][0].token
        total += 1
        if is_accurate(top_prediction=top_prediction, target=target):
            correct_connections[attr].append(sample_res)
            accurate += 1

print(f"Accuracy: {accurate/total}")

correct_connections["nationality"]

Accuracy: 0.7222222222222222


[{'query_entities': ('Fatima Sheikh', 'Mohammad Aziz'),
  'target': 'Pakistan',
  'connection': 'Pakistani',
  'next_tok_probs': [PredictedToken(token=' Pakistani', prob=0.2001953125, logit=18.5, token_id=45552, metadata=None),
   PredictedToken(token=' Indian', prob=0.1552734375, logit=18.25, token_id=7904, metadata=None),
   PredictedToken(token=' Turkish', prob=0.06494140625, logit=17.375, token_id=24666, metadata=None),
   PredictedToken(token=' Persian', prob=0.050537109375, logit=17.125, token_id=50961, metadata=None),
   PredictedToken(token=' French', prob=0.03466796875, logit=16.75, token_id=8753, metadata=None),
   PredictedToken(token=' Arabic', prob=0.03466796875, logit=16.75, token_id=35217, metadata=None),
   PredictedToken(token=' Iranian', prob=0.03466796875, logit=16.75, token_id=28501, metadata=None),
   PredictedToken(token=' Moroccan', prob=0.0306396484375, logit=16.625, token_id=95067, metadata=None),
   PredictedToken(token=' Afghan', prob=0.0306396484375, logit=1